# Tensor是什么
torch中的tensor类似numpy的ndarrays, 是最基本的数据结构，用于存储和操作多维数组。

In [ ]:
import torch
import numpy as np


___

### 创建tensor

In [ ]:
# 列表到张量
tensor_from_list = torch.tensor([1,2,30])
# ndarray 搭配张量
np_array = np.array([4,5,6])
tensor_from_numpy = torch.tensor(np_array)
print(tensor_from_numpy)

In [ ]:
# 全零
zero_tensor = torch.zeros((3, 4))
print(zero_tensor)
# 全1
ones_tensor = torch.ones((2,2))
print(ones_tensor)
# 指定范围张量
range_tensor = torch.arange(0, 10, 2)
print(range_tensor)
# 均匀分布的tensor
mean_tensor = torch.rand((3,3))
print(mean_tensor)
# 正态分布tensor
normal_tensor = torch.randn((3,3))
print(normal_tensor)

In [ ]:
# 未初始化的tensor
empty_tensor = torch.empty((2,2))

# 常见与某个tensor形状相同的tensor
same_shape_tensor = torch.ones_like(normal_tensor)
print(empty_tensor)
print(same_shape_tensor)


___

## 张量的基本操作

In [ ]:
# 张量的尺寸
# tensor_from_list = torch.tensor([1,2,30])
shape_tensor = tensor_from_list.shape
print(shape_tensor)

# 索引
element = tensor_from_list[0]
print(element)

# 切片
sliced_tensor = tensor_from_list[:2]
print(sliced_tensor)

In [ ]:
# 改变张量的形状
reshaped_tensor = tensor_from_list.view(1,3)
# 改成1，3相当于从一维张量转为了二维张量
print(reshaped_tensor)
# 张量转置
transposed_tensor = tensor_from_list.t()
print(transposed_tensor)

**数学计算**

In [ ]:
tensor_o = torch.ones(1,3)
tensor_rand = torch.randn(1,3)
print(tensor_o, tensor_rand)

#加法操作
tensor_sum = tensor_o + tensor_rand
print("sum:", tensor_sum)

# 乘法操作: 注意matmul是典型的矩阵乘法
product_tensor = torch.matmul(tensor_sum, tensor_o.t())
print(product_tensor)
product_tensor_2 = torch.matmul(tensor_sum.t(), tensor_o)
print(product_tensor_2)

# 广播计算
tensor_o_broad = tensor_o * 3
print(tensor_o_broad)
tensor_o_broad_2 = tensor_o_broad + 2
print(tensor_o_broad_2)

## 自动求导


# 自动求导
pytorch自动求导的核心是torch.autograd模块，它为对张量的所有操作提供了自动求导服务，其核心部分包含:
+ tensor
+ 计算图
+ 梯度计算
+ 梯度累计
+ 离散跟踪

如果张量的requires_grad属性被设置为True，那么对它的每一步操作都将被跟踪。当结束计算后，可以调用**backward()**方法，来自动计算所有参数的梯度，注意这个梯度是**累加的**

In [ ]:
x = torch.randn(3, 3, requires_grad = True)
print(x.grad_fn)
# 由于x是用户创建的叶子节点，因此没有跟踪计算图

In [ ]:
x = torch.ones(2,2, requires_grad= True)
print(x)

In [ ]:
# 对张量进行一次计算
y = x ** 2
print(y)
print(y.grad_fn)
# more
z = y * y * 3
out = z.mean()
print(z, out)

由于y是对x进行操作的结果，因此这个操作会被跟踪，y的grad_fn属性为grad_fn=<PowBackward0>, 字面上看是幂计算反向传播。

In [ ]:
# requires_grad属性默认是false
a = torch.randn(3,2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
# .requires_grad_()方法可以原地修改张量的requires_grad属性
a.requires_grad_(True)
b = (a * a).sum()
print(b.grad_fn)

___

## 梯度

In [ ]:
# 对out反向传播
out.backward()

In [ ]:
# 看看d(out)/dx
print(x)
print(x.grad)

In [ ]:
# 雅可比向量积的例子:
j = torch.randn(3, requires_grad=True)
print(j)

k = j * 2
m = 0
while k.data.norm() < 1000:
    k = k * 2
    m = m + 1
print(k)
print(m)
# 此时k已经不再是标量，autograd不能直接计算梯度，因此只可以求雅可比向量积。
v = torch.tensor([0.1, 1.0, 0.0001],dtype=torch.float)
k.backward(v)
print(j.grad)

注意，auto_grad只能直接求**标量**结果的反向传播结果。如果结果是向量的话，对所有输入求导就会得到一个**雅可比矩阵**，而不是该输入的梯度值。因此，如果最终结果是向量的话，backward()需要引入一个与输出同维度的权重向量。

___

***梯度的累加机制***

In [ ]:
# 如果不手动对参数的梯度清零的话，其梯度会一直累加
out2 = x.sum()
out2.backward()
print(x.grad)
out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

___

## 离散追踪
推理或者验证模型的时候，我们不希望数值计算过程被追踪，因为那样会消耗额外的内存与算力。此时使用torch.no_grad()来临时将tensor的requires_grad属性设置为False。

In [ ]:
print(j.requires_grad)
print((j ** 2).requires_grad)

with torch.no_grad():
    print((j ** 2).requires_grad)

___
tip:如果我们希望改变一个tensor的值，却不想该百年它的autograd记录，那么就直接对tensor.data属性进行操作：

In [ ]:
x = torch.ones(1, requires_grad=True)

print(x.data)  # 其实还是一个tensor
print(x.data.requires_grad)

y = 2 * x
x.data *= 100 # 只改变了值，不会记录在计算图，所以不会影响梯度传播
y.backward()
print(x)
print(x.grad)  # 发现导数为2， 与100无关

___

## grad_fn属性详解

当我们对张量进行运算的时候，得到的张量的grad_fn会自动记录该操作，**并指向一个Function对象**，该对象复制前向传播和反向传播。tensor与Function共同组成一个无环有向的计算图。



# 多gpu并行计算

### 什么是CUDA?
CUDA是英伟达GPU的并行运算框架。对GPU的编程是使用CUDA语言完成的。然而pytorch的cuda意思是我们即将使用GPU来处理数据和模型。

当我们希望把模型或者数据从cpu迁移到gpu时，就需要使用.cuda()方法（默认0号gpu）。
tips：
+ pytorch目前不支持amd的opengl接口
+ 避免频繁地将数据在cpu和gpu之间切换
+ 进行简单操作时，使用cpu

——————

如何设置默认显卡？两种方法


In [ ]:
 #设置在文件最开始部分
import os
os.environ["CUDA_VISIBLE_DEVICE"] = "2" # 设置默认的显卡
# 或者
CUDA_VISBLE_DEVICE=0,1 python train.py # 使用0，1两块GPU

## 常见的并行训练方法：

**1.mambaout,模型拆分**  
![模型拆分图解](https://datawhalechina.github.io/thorough-pytorch/_images/model_parllel.png)  
该方法难在gpu之间的通信

**2.同一层任务分不到不同数据中**
这种方法将同一层模型做拆分，不太理解  

**3.不同数据拆分到不同设备(数据并行方式, data parallelism)**  
![](https://datawhalechina.github.io/thorough-pytorch/_images/data_parllel.png)  
该方法的逻辑是将相同的模型复制到各个显卡中，然后将数据切分，让各个显卡训练数据的一部分，最后进行汇总反向传播。这样可以解决通信问题。

___


### 多卡训练 
pytorch提供了data parallel(DP)方式和DistributedDataParallel(DDP)两种多卡训练方式

***使用CUDA加速训练***  
**单卡训练**  
非常简单，只要将模型和数据.cuda()就行了  
首先要手动指定对程序可见的gpu

**多卡DP**

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"

In [ ]:
model = Net()
model.cuda() # 模型显示转移到CUDA上

if torch.cuda.device_count() > 1: # 含有多张GPU的卡
	model = nn.DataParallel(model) # 单机多卡DP训练

DP的抽象过程，数据被分为多个子集后，分别在各个gpu计算梯度，最后把梯度汇总到一个主gpu上进行参数的更新，这样会使得主gpu的工作负担明显大于其他gpu，成为性能瓶颈。此外，DP只适合单机模式。**注意DP是单进程多线程**

问题：既然python解释器有GIL锁，为什么限制不了DP的加速？
因为pytorch数据运行于gpu上，由CUDA和其他GPU加速库管理，不受python解释器管理。

**多机多卡DDP**  
DDP是一种多卡多进程方式，每个进程独立运行在一个gpu上，每个gpu独立更新参数，使用高效的通信机制进行梯度信息同步。

## GIL锁对于DP与DDP的影响？
由于模型训练大部分内容是在gpu上运行的，因此对于DP，GIL锁对工作的大部分内容几乎没有影响。
但是对于**运行于cpu的数据预处理加载和IO操作，还有数据后处理和日志记录等**，可能成为性能瓶颈。
但是对于多进程的DDP，由于各个进程都有独立的python解释器，实现了真正的并行运行，因此不受GIL影响。

这对于严重依赖 Python runtime 的 models 而言，比如说包含 RNN 层或大量小组件的 models 而言，这尤为重要。什么是python runtime？？？

# 深度学习训练的整体流程
+ 数据预处理： 数据格式统一、异常数据清除、数据变换
+ 划分训练集、验证集、测试集（可以使用sklearn自带的test_train_split函数）
+ 模型选择、损失函数、优化方法
+ 超参数设置
+ 使用模型拟合训练数据集，在验证集、测试机上评估模型表现



## 包的导入

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optimizer

In [ ]:
# 显式指定gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1' #指明gpu为01
device = torch.device("cuda:1" if torch.cuda.is_available else cpu)
print(device)

In [ ]:
# 设置一些超参数
batch_size = 16

lr = 1e-4

epoch = 100
# 当然我们可以选择把超参数存储在yaml里面

## 数据读入
+ dataset定义数据格式和数据变形形式
+ dataloader以迭代的方式，向模型输入批次数据

___

## dataset
**使用torch自建的dataset ImageFolder**

In [ ]:
# 以cifar10数据集构建Dataset类的方式
import torch
from torchvision import datasets
# 这里使用了PyTorch自带的ImageFolder类的用于读取按一定结构存储的图片数据（path对应图片存放的目录，目录下包含若干子目录，每个子目录对应属于同一个类的图片）
train_data = datasets.ImageFolder(train_path, transfor=data_transform)
val_data = datasets.ImageFolder(val_path, transform=data_transform)

**使用自定义的dataset**

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class MyDataset(Dataset):
    # 注意实现Dataset父类
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        """
        image1.jpg, 0
        image2.jpg, 1
        ......
        image9.jpg, 9
        
        """
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        # 数据加载
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

构建完成dataset以后，就可以使用dataloader批次读入数据了。

In [ ]:
import torch.utils
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=4, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=4, shuffle=False)
# 参数解析
# num_workers, cpu读取数据的进程数
# drop_last, 对于最后达不到batch_size数量的样本，是否丢弃？

___

# 模型构建
**Module 类**是 torch.nn 模块里提供的一个模型构造类，是所有神经网络模块的基类，我们可以继承它来定义我们想要的模型。



In [7]:
# 使用nn.Module构建多层感知机
import torch
from torch import nn

class MLP(nn.Module):
    # 在__init__中构建神经网络
    def __init__(self, **kwargs):
    # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784,256)
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)
    
    # 定义模型的前向计算
    def forward(self, x):
        o = self.act(self.hidden(x))
        return self.output(o)

In [8]:
# 随机设置一个输入
x = torch.randn(2, 784) # 第一维是batch
net = MLP()
print(net)
net(x)  # 前向计算

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[ 0.1897, -0.2245, -0.0114, -0.3305,  0.2267, -0.1157,  0.2367, -0.0760,
          0.0160,  0.0921],
        [-0.0816, -0.0649,  0.1001, -0.0160,  0.1412, -0.2122, -0.3227, -0.3230,
         -0.2738, -0.1120]], grad_fn=<AddmmBackward0>)

### 深度学习有各种各样的层, 本节学习使用torch构建自定义层

In [9]:
# 不含参数的层
import torch
from torch import nn

class MyLayer(nn.Module):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()  

In [10]:
# 自定义参数的层，注意Parameter是Tensor的子类型，会被自动添加到模型的参数列表之中,使之可以训练
# 这样的话，似乎就可以实现模型手撕了
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))

    def forward(self, x):
        for i in range(len(self.params)):
            # mm，仅仅可以对付二维矩阵乘法
            x = torch.mm(x, self.params[i])
        return x
net = MyListDense()
print(net)

class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                'linear1': nn.Parameter(torch.randn(4, 4)),
                'linear2': nn.Parameter(torch.randn(4, 1))
        })
        self.params.update({'linear3': nn.Parameter(torch.randn(4, 2))}) # 新增

    def forward(self, x, choice='linear1'):
        return torch.mm(x, self.params[choice])

net = MyDictDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.float32 of size 4x4]
      (1): Parameter containing: [torch.float32 of size 4x4]
      (2): Parameter containing: [torch.float32 of size 4x4]
      (3): Parameter containing: [torch.float32 of size 4x1]
  )
)
MyDictDense(
  (params): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


___

### 卷积层

In [11]:
# 手撕二维卷积！！！

import torch
from torch import nn

def corr2d(X, K):
    h, w = K.shape
    print(h,w)
    X, K = X.float(), K.float()
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init()
        # super 方法用于为子类调用父类方法，type参数
        # 传入类型， obj传入实例，在类定义中, obj传入
        # self
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [13]:
# 使用padding进行same卷积
import torch
from torch import nn

def comp_conv2d(conv2d, X):
    X = X.view((1,1) + X.shape) # 为X加入批次和通道维度
    Y = conv2d(X)
    return Y.view(Y.shape[2:]) # 滤除批次和通道维度

# 对输入左右padding两行
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding= 1)

X = torch.rand(8,8)
# 注意卷积之后前后变换公式: y = (x - h + 2 * p)/s + 1, 注意向下取整
print(comp_conv2d(conv2d, X).shape)

# 使用高宽不一致的卷积核
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5, 3), padding=(2, 1))
print(comp_conv2d(conv2d, X).shape)

# 引入stride
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(3, 5), padding= (0, 1), stride = (3, 4))
print(comp_conv2d(conv2d, X).shape)



torch.Size([8, 8])
torch.Size([8, 8])
torch.Size([2, 2])


In [17]:
# 手撕池化层
import torch
from torch import nn

def pool2d(X, pool_size, mode = 'max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

X = torch.tensor([[0,1,2],[4,5,6],[7,8,9]], dtype = torch.float)
print(pool2d(X, (2,2)))
print(pool2d(X,(2,2), "avg"))

tensor([[5., 6.],
        [8., 9.]])
tensor([[2.5000, 3.5000],
        [6.0000, 7.0000]])


## 卷积层的计算量
**输出维度**：假设输入特征的shape为H * W * C， 卷积核大小为h * w, 数量为c， 步长为s， padding为n，求输出的shape：
$$\left(\left\lfloor \frac{H - h + 2n}{s} \right\rfloor + 1, \left\lfloor \frac{W - w + 2n}{s} \right\rfloor + 1, c \right)$$
**总计算量**: 在上述背景下，所需的总计算量：  
从输出角度看比较简单理解, 设输出维度分别是$H_o$, $W_o$, 输出的每一个数据点所需要的乘法是$H_o * W_o * h * w * C * c$，加法操作是$H_o * W_o * C * (h * w * c - 1)$
即$$\text{乘法计算量} = \left( \left\lfloor \frac{H - h + 2n}{s} \right\rfloor + 1 \right) \times \left( \left\lfloor \frac{W - w + 2n}{s} \right\rfloor + 1 \right) \times h \times w \times C \times c$$
**感受野**：卷积神经网络的感受野指的是某层神经网络的神经元能感受到输入层的特征数量。  
感受野计算公式：
$$R_{l+1} = R_l + (k - 1) \cdot \prod_{i=1}^{l} s_i$$ 
因此感受野受到以下因素影响：
+ 池化可以使得数据分辨率降低，使得后序层的感受野增大
+ 卷积核宽度，宽度越宽，感受野越大
+ 步长，更大的步长可以增加感受野，但也会使得特征图分辨率下降加速
+ padding
+ 神经网络深度
+ 膨胀卷积

___

### 关于U-NET中上采样或者解码层的一些补充
上采样的两种方法：
+ 双线性插值上采样(无需学习): 使用插值的方法增加像素点，通过双线性插值可以得到平滑输出但是
失去高频细节
+ 转置卷积(需要学习)

**转置卷积的实现：** 首先是输入扩展，在每个元素周围插入stride-1个0，然后使用常规的卷积方式进行卷积。  
最终输出尺寸是:
$$H_{\text{out}} = s \cdot (H - 1) + k - 2p$$
$$W_{\text{out}} = s \cdot (W - 1) + k - 2p$$
其中k是卷积核尺寸，p是padding。


## RNN
![](RNN.png)

pytorch 中使用 nn.RNN 类来搭建基于序列的循环神经网络，它的构造函数有以下几个参数：

- input_size：输入数据X的特征值的数目。
- hidden_size：隐藏层的神经元数量，也就是隐藏层的特征数量。在torch中也是该RNN输出特征数量。
- num_layers：循环神经网络的层数，默认值是 1。
- bias：默认为 True，如果为 false 则表示神经元不使用 bias 偏移参数。
- batch_first：如果设置为 True，则输入数据的维度中第一个维度就是 batch 值，默认为 False。默认情况下第一个维度是序列的长度， 第二个维度才是 - - batch，第三个维度是特征数目。
- dropout：如果不为空，则表示最后跟一个 dropout 层抛弃部分数据，抛弃数据的比例由该参数指定。

在原始RNN中，记忆的传递仅仅依赖于隐藏状态，计算公式如下：
$$h_t = f(W_{hx}x_t + W_{hh}h_{t-1} + b_h)$$
$$\hat{y}_t = g(W_{yh}h_t + b_y)$$
注意，RNN每个时间步共享一个RNN单元，意味着参数W和b在每个时间步都是不变的。  
因此，RNN可以认为是一个带有记忆传递的全连接层。

**总的计算量**: 在ht的计算过程中，不计激活函数的话，两个矩阵乘法所需要的乘法数量为$2K^2$, 需要的加法数量为$2K(K-1)$, 此外还要计入与偏执的2K次加法，因此总的计算量为$4K^2$

___

### RNN的梯度消失与梯度爆炸问题
RNN的损失为每个时间步损失的总和：
![](https://pic3.zhimg.com/v2-54298417fcad6982932fbf3164a4c29a_r.jpg)
其反向传播使用跨越时间反向传播算法(BPTT)  
$$\frac{\partial \mathcal{L}}{\partial \mathbf{W}} = \sum_{t=1}^{T} \frac{\partial \mathcal{L}_t}{\partial \mathbf{W}} = \sum_{t=1}^{T} \frac{\partial \mathcal{L}_t}{\partial \mathbf{y}_t} \frac{\partial \mathbf{y}_t}{\partial \mathbf{h}_t} \underbrace{\frac{\partial \mathbf{h}_t}{\partial \mathbf{h}_k}}_{*} \frac{\partial \mathbf{h}_k}{\partial \mathbf{W}}$$
其中  
$$\frac{\partial \mathbf{h}_t}{\partial \mathbf{h}_k} = \frac{\partial \mathbf{h}_t}{\partial \mathbf{h}_{t-1}} \frac{\partial \mathbf{h}_{t-1}}{\partial \mathbf{h}_{t-2}} \cdots \frac{\partial \mathbf{h}_{k+1}}{\partial \mathbf{h}_k} = \prod_{i=k+1}^{t} \frac{\partial \mathbf{h}_i}{\partial \mathbf{h}_{i-1}}$$
这个雅可比矩阵可以进一步分解，不同时间步的隐藏状态计算公式共享一个W
$$\prod_{i=k+1}^{t} \frac{\partial \mathbf{h}_i}{\partial \mathbf{h}_{i-1}} = \prod_{i=k+1}^{t} \mathbf{W}^\top \operatorname{diag} \left[ f'(\mathbf{h}_{i-1}) \right]$$
对W进行矩阵分解
$$\mathbf{W} = \mathbf{V} \operatorname{diag}(\lambda)\mathbf{V^{-1}}$$
由此可知，t个W相乘以后，如果W的最大特征值大于1，会导致梯度爆炸，最大特征值小于1，会导致梯度消失。因此为了抑制梯度消失与梯度爆炸问题，每次BPPT只进行固定的时间步数，从而导致了RNN无法处理长序列记忆问题。  
此外也可以采取梯度裁剪来抑制梯度爆炸。  
梯度消失问题稍微难解决问题，最容易理解的就是直接使用LSTM或RNN。

## LSTM
LSTM解决了RNN只能解决短期依赖关系的问题，加入门控机制，来实现对记忆信息的控制，达到选择性遗忘或记忆之前的内容，从而学会长期依赖。 
![](https://pic1.zhimg.com/v2-2033db91e70559f363eca4bf365b8c44_r.jpg) 



LSTM包含三个门：
+ 输入门:
+ 输出门
+ 遗忘门  
这三个门的计算公式一致，只是参数独立: 
$$\mathbf{i}_t = \sigma(\mathbf{W}_i \mathbf{x}_t + \mathbf{U}_i \mathbf{h}_{t-1} + \mathbf{b}_i)$$
$$\mathbf{f}_t = \sigma(\mathbf{W}_f \mathbf{x}_t + \mathbf{U}_f \mathbf{h}_{t-1} + \mathbf{b}_f)$$
$$\mathbf{o}_t = \sigma(\mathbf{W}_o \mathbf{x}_t + \mathbf{U}_o \mathbf{h}_{t-1} + \mathbf{b}_o)$$

此外，LSTM相比于RNN多加入一个记忆单元(cell)，它的作用是保持t时刻的一个关键信息，并使之可调控地保存一段时间。
候选cell的更新公式如下  
$$\tilde{\mathbf{c}_t} = \operatorname{tanh}(\mathbf{W}_o \mathbf{x}_t + \mathbf{U}_o \mathbf{h}_{t-1} + \mathbf{b}_o)$$
遗忘门、输入门的作用是决定候选记忆单元与t-1时间步记忆单元之间的混合比例，并形成当前时间步的记忆单元，输出门的作用是如何把当前记忆单元映射为当前隐藏状态(输出门决定当前多少记忆单元要输出为隐藏状态)。  
forget gate | input gate |  result  

 1  |  0  |  保留上一时刻的状态   
 1  |  1  |  保留上一时刻和添加新信息   
 0  |  1  |  清空历史信息，引入新信息   
 0  |  0  |  清空所有新旧信息    
 这样看的话，举一个极端案例，如果遗忘门始终为1，输入门始终为0，那么就无论多少时间步，最初的记忆总能影响到最终结果。
 ### 为什么LSTM能抑制梯度消失问题呢？  
 因为LSTM已经不再直接把隐藏状态作为记忆传递，而是通过一个可调控的记忆单元cell来传递记忆，从而使得梯度逆时间步流动时，可以直接通过遗忘门与输入门回到更早时间步，从而避免了梯度消失。

## GRU


LSTM有三个门，以及记忆单元计算，这导致了大量的权重矩阵需要学习，降低了训练速度。因此基于LSTM进行简化得到了GRU，它简化了LSTM输入门与遗忘门之间的互补关系，并取消了记忆单元，如同RNN一样直接使用隐藏状态传递记忆。  
![](https://pic2.zhimg.com/v2-6a83a3f4783bddb3479436724aa9150d_r.jpg)  
如图，GRU只要两个门：
+ 重置reset门
+ 更新update门
重置门用于生成候选隐藏状态，更新门以一定比例混合候选隐藏状态和上一步的隐藏状态。
$$\begin{align*}
r_t &= \sigma(\mathbf{W}_r \mathbf{x}_t + \mathbf{U}_r \mathbf{h}_{t-1} + \mathbf{b}_r) \\
z_t &= \sigma(\mathbf{W}_z \mathbf{x}_t + \mathbf{U}_z \mathbf{h}_{t-1} + \mathbf{b}_z) \\
\tilde{\mathbf{h}}_t &= \tanh(\mathbf{W}_h \mathbf{x}_t + r_t \odot (\mathbf{U}_h \mathbf{h}_{t-1}) + \mathbf{b}_h) \\
\mathbf{h}_t &= z_t \odot \mathbf{h}_{t-1} + (1 - z_t) \odot \tilde{\mathbf{h}}_t
\end{align*}
$$
观察公式发现，当重置门的值为1的时候，GRU与RNN抑制。更新门数值为1的时候，改GRU就完全变成记忆传递通道，与当前输入无关了。


___

# 模型初始化

### 为什么参数初始化不能全部为0  
当网络中所有权重都初始化为0时，对于任何隐藏层，所有神经元在前向传播和反向传播过程中的行为都会完全相同。这意味着每一层的每个神经元将计算出相同的输出，并且在反向传播中也会接收到相同的梯度。结果是，即使网络的参数被更新，每个权重也会保持相同，这使得隐藏层的多个神经元没有区分开来，因此无法学习到多样化的特征或模式。  
此外，对于一些过零的激活函数（如Sigmoid或Tanh），全部初始化为0也会导致梯度消失问题，当激活函数的输入接近0的时候，其反向传播获得的梯度也会接近0。

**nn.init中提供了一些初始化的方法，这里直接复制**
 1 . torch.nn.init.uniform_(tensor, a=0.0, b=1.0)   
 2 . torch.nn.init.normal_(tensor, mean=0.0, std=1.0)   
 3 . torch.nn.init.constant_(tensor, val)   
 4 . torch.nn.init.ones_(tensor)   
 5 . torch.nn.init.zeros_(tensor)   
 6 . torch.nn.init.eye_(tensor)   
 7 . torch.nn.init.dirac_(tensor, groups=1)   
 8 . torch.nn.init.xavier_uniform_(tensor, gain=1.0)   
 9 . torch.nn.init.xavier_normal_(tensor, gain=1.0)   
 10 . torch.nn.init.kaiming_uniform_(tensor, a=0, mode='fan__in', nonlinearity='leaky_relu')   
 11 . torch.nn.init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')   
 12 . torch.nn.init.orthogonal_(tensor, gain=1)   
 13 . torch.nn.init.sparse_(tensor, sparsity, std=0.01)   
 14 . torch.nn.init.calculate_gain(nonlinearity,  
 方法带有下划线，意味着这些方法能原地改变张量的值

In [ ]:
# isinstance方法可以判定某个实例是不是属于某个类型
import torch
import torch.nn as nn

conv = nn.Conv2d(1,3,3)
linear = nn.Linear(10,1)


print(isinstance(conv, nn.Conv2d))
print(isinstance(linear, nn.Conv2d))

In [ ]:
# 查看一波层的参数
print(conv.weight.data)
# 3*3的卷积核，输出通道为3，似乎创建出来就被自动初始化了
print(linear.weight.data)

In [ ]:
# 对conv进行kaiming初始化
torch.nn.init.kaiming_normal_(conv.weight.data)

In [ ]:
# 对linear进行常数初始化
torch.nn.init.constant_(linear.weight.data,0.3)

___

# 损失函数
损失函数衡量了预测信息与标签信息之间的差距，用于为模型训练提供负反馈，是反向传播的起点。



___

### PMSQE损失：基于感知指标的语言质量评估
传统的均方误差损失MSE只考虑预测语音与目标语音之间的功率谱之间的均方误差，仅仅在数学上缩小了二者功率谱的差异，没有考虑人耳对声音的感知特性，因此基于MSE训练出来的模型在听觉上往往不如人意。
PMSQE在MSE的基础上，增添了两个感知扰动项：对称扰动与非对称扰动。它们是基于PESQ简化而来，其中对称扰动衡量了预测语音与目标语音之间的响度差异;非对称扰动是对对称扰动进行加权得到的，加权的值取决于对称扰动的正负，因为人耳对音量增大的敏感度高于对音量减小的敏感度，正差异会获得更大损失。
最终MSE、对称扰动、非对称扰动共同构成了PMSQE损失。

___
### 基于pase编码器的损失：
PASE是一个使用子监督方法训练得到的语音特征提取器，其提取出来的语音特征被证明比传统的MFCC谱更有更好性能。基于PASE模型得到音频的embedding，计算预测语音与目标语音embedding之间的MSE。

___

# 优化器
优化器是根据网络反向传播的梯度信息来**更新网络的参数**，以起到降低loss函数计算值，使得模型输出更加接近真实标签。


**torch提供了一个优化器库torch.optim**提供多种优化器
torch.optim.SGD

torch.optim.ASGD

torch.optim.Adadelta

torch.optim.Adagrad

torch.optim.Adam
...
这些优化器都继承自基类**Optimizer**,定义:

In [1]:
class Optimizer(object):
    def __init__(self, params, defaults):        
        self.defaults = defaults # 存储一些通用超参数，比如momentum的beta，rmsprop的beta
        self.state = defaultdict(dict) # 优化器状态参数缓存，比如累加的梯度
        self.param_groups = [] # 字典列表，每个元素是个字典，包含需要训练的parameter，对应的学习率，权重衰减

    # 介绍Optimizer的一些方法
    # 清空梯度缓存
    def zero_grad(self, set_to_none: bool = False):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is not None:  #梯度不为空
                    if set_to_none: 
                        p.grad = None
                    else:
                        if p.grad.grad_fn is not None:
                            p.grad.detach_()  # 断开计算图
                        else:
                            p.grad.requires_grad_(False)
                        p.grad.zero_()# 梯度设置为0

    # 执行参数更新
    def step(self, closure): 
        raise NotImplementedError

    # 添加参数组
    def add_param_group(self, param_group):
        pass

    # 加载状态参数字典，用于断点续训练
    def load_state_dict(self, state_dict):
        ...

    def cast(param, value):
        ...

    # Update parameter groups, setting their 'params' value
    def update_group(group, new_group):
       ...
       
    # 获取当前优化器状态信息字典
    def state_dict(self):
        ...

    def pack_group(group):
        ...


### 随机梯度下降法SGD
在torch中，欲实现SGD，需要设置batch_size为1。即计算一个样本的损失，就更新参数。
+ 缺陷：噪声与抖动非常大，而且永远无法收敛至最小值。

### mini-batch梯度下降法
将m个样本拼接成一个batch，累加m个样本的梯度求平均，然后更新参数，噪声相对于SGD小。当然性能消耗也会大于SGD。

### batch梯度下降法
将所有样本一次性输入，累加梯度后求平均，噪声非常小，但数据量巨大，在现代深度学习任务中几乎不可能实现。

## 知识点：指数加权平均
指数加权平均是一种计算机易实现的数据平滑处理。
$v_t = \beta v_{t-1} + (1 - \beta)\theta_t$  
v和theta分别代表了平滑值与瞬时值。平滑窗口长度约为$\frac{1}{1 - \beta}$

### momentum动量梯度下降法：
momentum的核心在于指数加权平均，在梯度下降过程中，正负摆动会被平滑掉，而朝着损失下降的方向由于固定，不会被平滑。在momentum梯度下降法中，超参数包含$\beta$与$\alpha$，分别是指数加权平均的参数和学习率。

### RMSprop梯度下降算法：
不同于momentum仅仅平滑了梯度，RMSprop还对参数变化进行了加速。
$$S_{dw} = \beta S_{dw} + (1 - \beta)dW^2$$
$$S_{db} = \beta S_{db} + (1 - \beta)db^2$$
$$W = W - \alpha \frac{dW}{\sqrt{S_{dW}}}$$
$$W = b - \alpha \frac{db}{\sqrt{S_{db}}}$$  

对于变化比较剧烈的参数，S会比较大，更新时会收到抑制，变化微弱的参数，更新时会得到放大。

### Adam优化算法
adam融合了momentum算法和RMSprop算法，既对梯度进行了平滑，又对参数更新进行了加速。因此同时拥有了动量梯度下降的$\beta1$，又有了RMSprop算法$\beta2$

___

## torch优化器实际操作

In [1]:
import os
import torch

# 初始化权重
weight = torch.randn((2,2), requires_grad=True)
# 设置梯度为全1矩阵 --> 2 * 2
weight.grad = torch.ones((2,2))
print("The data of weight before step:\n{}".format(weight.data))
print("The grad of weight before step:\n{}".format(weight.grad))

The data of weight before step:
tensor([[-0.5980, -2.6022],
        [-0.2394,  0.7446]])
The grad of weight before step:
tensor([[1., 1.],
        [1., 1.]])


In [2]:
# 实例化优化器
optimizer = torch.optim.SGD([weight], lr=0.1, momentum=0.9)
# 更新下参数
optimizer.step()
print("The data of weight after step:\n{}".format(weight.data))
print("The grad of weight after step:\n{}".format(weight.grad))
# 发现参数都降低了0.1

The data of weight after step:
tensor([[-0.6980, -2.7022],
        [-0.3394,  0.6446]])
The grad of weight after step:
tensor([[1., 1.],
        [1., 1.]])


In [3]:
# 梯度清零
optimizer.zero_grad()
print("The grad of weight after optimizer.zero_grad():\n{}".format(weight.grad))

The grad of weight after optimizer.zero_grad():
None


In [4]:
print("optimizer.params_group is \n{}".format(optimizer.param_groups))
print("weight in optimizer:{}\nweight in weight:{}\n".format(id(optimizer.param_groups[0]['params'][0]), id(weight)))
# 上面发现optimizer.param_groups[0]["params"[0]]和weight指向同一个对象

optimizer.params_group is 
[{'params': [tensor([[-0.6980, -2.7022],
        [-0.3394,  0.6446]], requires_grad=True)], 'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}]
weight in optimizer:140464894949392
weight in weight:140464894949392



In [5]:
# 添加参数：weight2
weight = torch.rand((3,3 ), requires_grad= True)
optimizer.add_param_group({"params": weight, "lr" :0.0001, "nesterov": True})
# 查
print("optimizer.param_groups is\n{}".format(optimizer.param_groups))
# 发现default属性的确存储的是统一超参数，但优先级低于param_group中的超参数

optimizer.param_groups is
[{'params': [tensor([[-0.6980, -2.7022],
        [-0.3394,  0.6446]], requires_grad=True)], 'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}, {'params': [tensor([[0.4579, 0.4673, 0.5956],
        [0.7286, 0.6144, 0.7743],
        [0.4208, 0.2964, 0.2083]], requires_grad=True)], 'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}]


In [6]:
# 查看优化器当前状态信息
opt_state_dict = optimizer.state_dict()
print("state_dict before step:\n", opt_state_dict)

state_dict before step:
 {'state': {0: {'momentum_buffer': tensor([[1., 1.],
        [1., 1.]])}}, 'param_groups': [{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'params': [0]}, {'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'params': [1]}]}


In [8]:
# 50次step
for _ in range(50):
    optimizer.step()
print("state_dict after step:\n", optimizer.state_dict())
# 注意state中只存储一些当前的超参数、梯度等信息，并不存储data
# 保存参数信息
torch.save(optimizer.state_dict(),os.path.join(r"D:\pythonProject\Attention_Unet", "optimizer_state_dict.pkl"))
print("----------done-----------")
# 加载参数信息
state_dict = torch.load(r"D:\pythonProject\Attention_Unet\optimizer_state_dict.pkl") # 需要修改为你自己的路径
optimizer.load_state_dict(state_dict)
print("load state_dict successfully\n{}".format(state_dict))
# 输出最后属性信息
print("\n{}".format(optimizer.defaults))
print("\n{}".format(optimizer.state))
print("\n{}".format(optimizer.param_groups))

state_dict after step:
 {'state': {0: {'momentum_buffer': tensor([[1., 1.],
        [1., 1.]])}}, 'param_groups': [{'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'params': [0]}, {'lr': 0.0001, 'nesterov': True, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'params': [1]}]}


In [ ]:
___

# 模型定义
nn.Module是torch中所有网络模块的基类型。



### 使用Sequential类来定义串联层

In [11]:
# 直接法
import torch.nn as nn
net = nn.Sequential(
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256,10)
)
print(net)

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [12]:
# 使用OrderedDict
import collections
import torch.nn as nn
net2 = nn.Sequential(collections.OrderedDict([
          ('fc1', nn.Linear(784, 256)),
          ('relu1', nn.ReLU()),
          ('fc2', nn.Linear(256, 10))
          ]))
print(net2)

Sequential(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


### 模型列表ModuleList
接受子模块作为输入，可以类似list那样append和extend

In [13]:
net = nn.ModuleList([nn.Linear(784,256), nn.ReLU()])
net.append(nn.Linear(256,10))
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


**注意modulelist不能像sequential那样直接forward，而应该遍历列表的层分别forward**

# 修改现有模型
开源模型越来越多，我们没必要手动去搓模型了。

### 篡改resnet50

In [15]:
import torch
import torch.nn as nn
from collections import OrderedDict
import torchvision.models as models
net = models.resnet50()
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

如果我们希望修改model，进行十分类

In [16]:
# 我们把最终的fc层修改为含有两个隐藏层的全链接层
classifier = nn.Sequential(OrderedDict([
    ("fc1", nn.Linear(2048, 128)),
    ('relu1', nn.ReLU()), 
    ('dropout1',nn.Dropout(0.5)),
    ('fc2', nn.Linear(128, 10)),
    ('output', nn.Softmax(dim=1))
]))
net.fc = classifier
print(net)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

___

# torch中模型的输出&存储


torch的模型一般以pkl,pt或者pth三种格式存储.pt和pth必须有pytorch环境的支持,pkl更加通用而不安全.  
pt和pth本质上是一样的,用于存储pytorch的模型和张量.  
pkl是python的通用数据序列化格式,由python的pickle模块实现,可以序列化几乎所有python对象.

In [ ]:
from torchvision import models
model = models.resnet152(pretrained=True)
save_dir = './resnet152.pth'

# 保存整个模型
torch.save(model, save_dir)
# 保存模型权重
torch.save(model.state_dict, save_dir)

# 动态调整学习率
学习率过小会造成学习缓慢，过大会导致损失震荡，因此，我们需要动态调整学习率，训练初期我们希望训练更加快速，因此学习率较大，后期为了防止损失震荡，会降低学习率。在pytorch中，可以设置一个学习率衰减策略来动态调整学习率，这个工具叫做调度器scheduler。


在torch.optim.lr_scheduler模块中封装好了一系列调度器：
lr_scheduler.LambdaLR

lr_scheduler.MultiplicativeLR

lr_scheduler.StepLR

lr_scheduler.MultiStepLR

lr_scheduler.ExponentialLR

lr_scheduler.CosineAnnealingLR

lr_scheduler.ReduceLROnPlateau

lr_scheduler.CyclicLR

lr_scheduler.OneCycleLR

lr_scheduler.CosineAnnealingWarmRestarts

lr_scheduler.ConstantLR

lr_scheduler.LinearLR

lr_scheduler.PolynomialLR

lr_scheduler.ChainedScheduler

lr_scheduler.SequentialLR

他们都继承自_LRScheduler类

In [ ]:
# 学习率调度往往是在训练完一个epoch后执行的：
# 选择一种优化器
optimizer = torch.optim.Adam(...) 
# 选择上面提到的一种或多种动态调整学习率的方法
scheduler1 = torch.optim.lr_scheduler.... 
scheduler2 = torch.optim.lr_scheduler....
...
schedulern = torch.optim.lr_scheduler....
# 进行训练
for epoch in range(100):
    train(...)
    validate(...)
    optimizer.step()
    # 需要在优化器参数更新之后再动态调整学习率
# scheduler的优化是在每一轮后面进行的
scheduler1.step() 
...
schedulern.step()

In [ ]:
# 在之前的内容中我们知道训练的权重、梯度、学习率等内容都存储在optimizer的param_group中
# 因此如果希望自定义一个scheduler，我们的核心就是修改param_group的"lr"
def adjust_learning_rate(optimizer, epoch):
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

optimizer = torch.optim.SGD(model.parameters(),lr = args.lr,momentum = 0.9)
for epoch in range(10):
    train(...)
    validate(...)
    adjust_learning_rate(optimizer,epoch)

# 深度学习模型微调
实际任务中，我们手上的数据很少，几乎不可能直接从头训练一个模型，因为可能会导致过拟合。因此借用别人的预训练模型来进行微调显得非常重要。
在pytorch中提供了非常多的预训练模型，比如VGG、Resnet等等。

### 微调的基本流程
+ 在开源大量的源数据集上训练一个源模型
+ 替换源数据集的输出层，使之适应新数据集的任务
+ 在目标数据集上，从头训练输出层，并微调其他层。  
![微调流程](https://datawhalechina.github.io/thorough-pytorch/_images/finetune.png)

In [3]:
# 首先，从torchvision库搞个预训练模型下来
# 默认情况下，models中的模型是不包含预训练参数的
import torchvision.models as models
resnet18 = models.resnet18(pretrained=False)
alexnet = models.alexnet()
vgg16 = models.vgg16()
squeezenet = models.squeezenet1_0()
densenet = models.densenet161()
inception = models.inception_v3()
googlenet = models.googlenet()
shufflenet = models.shufflenet_v2_x1_0()
mobilenet_v2 = models.mobilenet_v2()
mobilenet_v3_large = models.mobilenet_v3_large()
mobilenet_v3_small = models.mobilenet_v3_small()
resnext50_32x4d = models.resnext50_32x4d()
wide_resnet50_2 = models.wide_resnet50_2()
mnasnet = models.mnasnet1_0()

d:\miniconda\envs\project\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\miniconda\envs\project\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
d:\miniconda\envs\project\lib\site-packages\torchvision\models\inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
d:\miniconda\envs\project\lib\site-packages\torchvision\models\googlenet.py:47: FutureWarning: The default weight initialization of Go

In [8]:
# 因此需要显式地指明使用预训练参数
resnet18 = models.resnet18(pretrained=True)
# 如果出现下载不成功的话，可以手动下载权重，使用model.load_state_dicet方法来装载参数

### 训练模型的特定层

In [13]:
# 首先冻结所有参数的梯度计算
import torchvision.models as models
import torch.nn as nn

def set_param_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

feature_extract = True
model = models.resnet18(pretrained = True)
set_param_requires_grad(model, feature_extract)

# 自定义一个输出的全连接层
num_ftrs = model.fc.in_features
model.fc = nn.Linear(in_features = num_ftrs, out_features=4, bias=True)

#之后在训练过程中，model仍会进行梯度回传，但是参数更新则只会发生在fc层。
#通过设定参数的requires_grad属性，我们完成了指定训练模型的特定层的目标，这对实现模型微调非常重要。



d:\miniconda\envs\project\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\miniconda\envs\project\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 关于微调的一些经验
### Size一致的情况  
+ 新的数据集很少，但是跟源数据集分布相似，且尺寸一致的话，直接微调最终的fc层，冻结其他层。
+ 如果数据集数量少且与源数据集分布有差异，可以从神经网络中层开始微调。
+ 如果效果都不行的话，就把预训练模型的参数作为初始化重新训练
### Size不一致的情况  
+ 如果新的数据集尺寸跟原始数据集不一样，可以删掉最终的fc层，加入一些卷积和pool，迫使输出size一致，但这样不好
### 对网络进行重新训练  
+ 这种情况，可以尝试把中间层的学习率设置为预定的十分之一，输出层学习率设置为正常。

## 语音领域微调与CV领域微调的区别
+ 语音的底层逻辑差异极大，不同语言有不同的发音特征以及字词，但是语言表达的高级逻辑大致是相似的，因此往往微调浅层。
+ CV的底层逻辑差异不大，浅层神经网络往往负责点线面的切割，但高层信息处理的方式差异比较大，一般微调深层。


___

# 半精度训练
在pytorch中训练模型的过程中，默认的数据存储方式为torch.flat32, 这会使得计算精度更高，同时也导致了更大的算力和显存需求。  
因此在一些不要求那么精准的训练中，可以采样***半精度***训练，保留一半的数据长度，即使用float16。  
在torch中，我们使用autocast类来进行半精度配置。

In [14]:
from torch.cuda.amp import autocast

In [ ]:
# 在模型定义中，我们可以指定forward为半精度，使用autocast装饰。
@autocast()
def forward(self,x):
    ...
    return x

In [ ]:
# 在训练过程中，也可以在数据输入模型之前加入with autocast()上下文就可以了
for x in train_loader:
    x = x.cuda()
    with autocast():
        output = model(x)
    ...

___

# 特别学习：torchaudio，联想librosa、 soundfile



torchaudio.io：有关音频的I/O

torchaudio.backend：提供了音频处理的后端，包括：sox，soundfile等

torchaudio.functional：包含了常用的语音数据处理方法，如：spectrogram，create_fb_matrix等

torchaudio.transforms：包含了常用的语音数据预处理方法，如：MFCC，MelScale，AmplitudeToDB等

torchaudio.datasets：包含了常用的语音数据集，如：VCTK，LibriSpeech，yesno等

torchaudio.models：包含了常用的语音模型，如：Wav2Letter，DeepSpeech等

torchaudio.models.decoder：包含了常用的语音解码器，如：GreedyDecoder，BeamSearchDecoder等

torchaudio.pipelines：包含了常用的语音处理流水线，如：SpeechRecognitionPipeline，SpeakerRecognitionPipeline等

torchaudio.sox_effects：包含了常用的语音处理方法，如：apply_effects_tensor，apply_effects_file等

torchaudio.compliance.kaldi：包含了与Kaldi工具兼容的方法，如：load_kaldi_fst，load_kaldi_ark等

torchaudio.kalid_io：包含了与Kaldi工具兼容的方法，如：read_vec_flt_scp，read_vec_int_scp等

torchaudio.utils：包含了常用的语音工具方法，如：get_audio_backend，set_audio_backend等

In [ ]:
# 直接使用torchaudio自带的YESNO数据集构建dataset
import torchaudio
import torch

yesno_data = torchaudio.datasets.YESNO('.', download=True)
data_loader = torch.utils.data.DataLoader(
    yesno_data,
    batch_size=1,
    shuffle=True,
    num_workers=4)

In [15]:
# torchaudio自带了许多数据集
import torchaudio

dir(torchaudio.datasets)

['CMUARCTIC',
 'CMUDict',
 'COMMONVOICE',
 'DR_VCTK',
 'FluentSpeechCommands',
 'GTZAN',
 'IEMOCAP',
 'LIBRISPEECH',
 'LIBRITTS',
 'LJSPEECH',
 'LibriLightLimited',
 'LibriMix',
 'LibriSpeechBiasing',
 'MUSDB_HQ',
 'QUESST14',
 'SPEECHCOMMANDS',
 'Snips',
 'TEDLIUM',
 'VCTK_092',
 'VoxCeleb1Identification',
 'VoxCeleb1Verification',
 'YESNO',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'cmuarctic',
 'cmudict',
 'commonvoice',
 'dr_vctk',
 'fluentcommands',
 'gtzan',
 'iemocap',
 'librilight_limited',
 'librimix',
 'librispeech',
 'librispeech_biasing',
 'libritts',
 'ljspeech',
 'musdb_hq',
 'quesst14',
 'snips',
 'speechcommands',
 'tedlium',
 'utils',
 'vctk',
 'voxceleb1',
 'yesno']

### torchaudio的transforms
类似于torchvision的transforms继承于torch.nn.Module。下图为其方法图解，包含了一系列从频域到其他域的转变方法。  
![transforms包](https://datawhalechina.github.io/thorough-pytorch/_images/torchaudio_feature_extractions.png)

___

# 激活函数
如果没有激活函数的话，神经网络不管多复杂，最终的训练结果依然是一个线性函数，等效于只有一个隐藏层的fc网络。激活函数为深度学习网络引入非线性。

### Sigmoid函数
![](https://i-blog.csdnimg.cn/blog_migrate/2ac04d3695dd9563c3c9a4d357ed8103.jpeg)  
sigmoid适合用来做分类任务的输出层，但不太适合用于作为神经网络的激活函数
主要缺陷：  
+ 当输入数据较大或较小的适合，会导致梯度接近于0。这将导致梯度消失的问题
+ 由于simoid函数不过零，这会导致输入的x全为正或全为负时，会导致对权重的导数一直为正或负，导致神经网络训练无法收敛。
+ sigmoid中包含幂运算，耗时。


### tanh函数
![](https://i-blog.csdnimg.cn/blog_migrate/73191ced71b237087e4db602ef52037c.jpeg)  
tanh函数可以把数据压缩到-1 到 1范围内，可以克服sigmoid函数不过零的缺陷。但同时也会引入sigmoid梯度消失问题。本质上tanh函数是对sigmoid进行线性运算得到的。
tanh(x) = 2sigmoid(x) - 1

### ReLU函数
![](https://i-blog.csdnimg.cn/blog_migrate/fecf5213b4496be0e63e171c702794d1.jpeg)  
优点：  
+ 由于正方向导数都是1，因此模型收敛速度比sigmoid和tanh快，同时解决了梯度消失问题
+ 前向和反向的计算复杂度低

缺点：  
+ relu不是以零为中心的，依然可能导致模型不收敛
+ 神经元坏死：输出在复数部分，梯度永远为0，参数永远不更新。
+ 相对于sigmoid和tanh，relu不会对数据进行压缩

### LeakyRelu
相比于Relu，复数部分不为零，而是一个带有非常小坡度的直线。避免了relu神经元坏死线性。
![](https://i-blog.csdnimg.cn/blog_migrate/1320de0b64bd44540e0d39c926f187a0.jpeg)  
但性能未必强于Relu

# 后记，一些机器学习基本问题和解决方法：



## 高偏差与高方差
- 高偏差的解决方法：
    1. 增加隐藏层层数、神经元个数
    2. 使用更加复杂的模型
    3. 增加模型训练时间
- 高方差的解决方法：
    1. 使用L1 L2 正则化，缺点，参数难以寻找，计算成本高
    2. 引入dropout，随机神经元失活
    3. 加入更多的训练数据
    4. 训练早停：在训练过程中，参数w会越来越大，早停能防止w过大，实际上接近正则化效果，防止过拟合。早停的缺点是，不能同时降低损失与方差。

## 为什么在梯度下降过程中几乎不可能陷入局部最优解
当训练参数量非常少，比如二维参数的时候，很多时候会设想作图的多局部最优的目标函数。实际上，假设目标函数在某个方向是凹的或者凸的概率都是0.5，那么在深度学习解决这种解决高维函数的任务中，假设参数维度是20000，那么进入局部最优解的概率仅仅为$2^{-20000}$。事实上我们更容易陷入右边的鞍点！进入鞍点以后，梯度接近0，训练速度会很慢。adam算法能有效解决这个问题。

## BatchNorm与LayerNorm

### BatchNorm
在理解归一化之前，先粗略了解模型训练过程中的***内部协变量偏移***。协变量偏移指的是前一层参数的改变，会导致后层输入分布的改变，从而使得后层的权重变化不稳定，层与层之间的参数更新相互影响。最终训练出来的模型的泛化能力与推理能力也不强。  
批归一化解决的就是这个问题，他是在mini-batch层面上对数据进行归一化，计算mini-batch的均值和方差，然后对数据进行归一化。  
    ***优点***
+ 减少内部协变量偏移，增强各层之间的独立性。
+ 加快训练速度。可以使用更大学习率，因为数据都被归一化到一个比较小的范围。
+ 减少对初始参数的敏感性，因为每一层的输出最终都会被可学习的缩放和偏移因子处理。且不再需要额外设置bias了，因为最终都会被归一化处理。
+ 适用于大batch size的CNN  
***缺陷*** 
+ 对于batch size较小的情况，性能不好
+ 在RNN中效果不佳
+ 由于推理的时候，batch size可能与训练时不一样，甚至为1，因此必须以指数加权平均的方式，记录训练过程中得到的均值和方差，然后应用到推理中。

### layernorm
层归一化直接对每个数据样本求均值和方差，不需要额外记录均值与方差。比较适用于RNN的训练。

### 为什么归一化可以抑制梯度消失问题
+ 对于一些激活函数（如sigmoid或tanh），当输入值过大或过小时，其梯度会趋近于零。BatchNorm通过归一化将输入值保持在激活函数的有效区间内，从而保证了较大的梯度值。
+ 由于归一化减少了内部协变量偏移，因此梯度回传会变得更加平稳，较少了梯度消失的现象。
+ 减缓了由于参数初始化导致的梯度消失问题。

### 为什么会发生梯度消失and梯度爆炸？
因为在深层神经网络中，梯度计算是根据复合求导的链式法则来的，每一层神经网络的权重都会参与到梯度回传中，因此会导致梯度指数地减小或者增大，导致梯度消失或梯度爆炸。  
+ 导致梯度消失的原因：  
    + 如sigmoid和tanh这些带饱和值的激活函数，容易导致梯度消失（归一化）
    + 深层神经网络
    + 权重初始化，如果初始化值过大，就会导致层的输出过大，导致激活函数饱和。过小会导致层输出过小，导致梯度回传时，梯度很小。  
+ 导致梯度爆炸的原因： 
    + 使用没有饱和特征的激活函数，比如relu
    + 深层神经网络
    + 权重初始化过大，导致层输出过大，反向传播时导致梯度过大。  
    + 学习率过大，导致参数更新不稳定